In [1]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
import time
import math
from math import dist

In [2]:
model=YOLO('yolov8s.pt')

In [3]:
class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [7]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('veh2.mp4')


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0

tracker=Tracker()

cy1=322
cy2=368
offset=6
vh_down={}
vh_up={}
counter=[]
counter1=[]

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        
        # down side
        if cy1<(cy+offset) and cy1>(cy-offset):
            vh_down[id]=cy1
            
        if id in vh_down:
            if cy2<(cy+offset) and cy2>(cy-offset):
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter.count(id)==0:
                    counter.append(id)
                
        #up side
        if cy2<(cy+offset) and cy2>(cy-offset):
            vh_up[id]=cy2
            
        if id in vh_up:
            if cy1<(cy+offset) and cy1>(cy-offset):
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter1.count(id)==0:
                    counter1.append(id)
        
        

    cv2.line(frame,(274,cy1),(814,cy1),(255,255,255),1)
    cv2.putText(frame,('1line'),(274,318),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.line(frame,(177,cy2),(927,cy2),(255,255,255),1)
    cv2.putText(frame,('2ndline'),(181,363),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    #down count
    d=len(counter)
    cv2.putText(frame,('Going Down Side:- '+str(d)),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    #up count
    u=len(counter1)
    cv2.putText(frame,('Going up Side:- '+str(u)),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 1 person, 3 cars, 1 fire hydrant, 198.7ms
Speed: 2.0ms preprocess, 198.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 211.0ms
Speed: 4.0ms preprocess, 211.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 191.5ms
Speed: 3.0ms preprocess, 191.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 199.0ms
Speed: 3.0ms preprocess, 199.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 172.0ms
Speed: 2.0ms preprocess, 172.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 fire hydrant, 153.5ms
Speed: 2.0ms preprocess, 153.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 3 cars, 1 fire hydrant, 166.2ms
Speed: 2.0ms preprocess, 166.2ms inference,


0: 320x640 1 person, 2 cars, 118.0ms
Speed: 2.0ms preprocess, 118.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 148.5ms
Speed: 2.0ms preprocess, 148.5ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 161.0ms
Speed: 2.0ms preprocess, 161.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 157.0ms
Speed: 3.0ms preprocess, 157.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 160.1ms
Speed: 2.0ms preprocess, 160.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 161.1ms
Speed: 2.0ms preprocess, 161.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 144.1ms
Speed: 2.0ms preprocess, 144.1ms inferen

Speed: 3.0ms preprocess, 162.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 195.1ms
Speed: 2.0ms preprocess, 195.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 4 cars, 186.5ms
Speed: 2.0ms preprocess, 186.5ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 4 cars, 183.0ms
Speed: 3.0ms preprocess, 183.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 181.0ms
Speed: 2.0ms preprocess, 181.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 166.5ms
Speed: 2.0ms preprocess, 166.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 186.0ms
Speed: 2.0ms preprocess, 186.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 154.0ms
Speed: 2.0ms preprocess, 154.0ms inference, 2.0ms postproc


0: 320x640 2 cars, 2 fire hydrants, 1 stop sign, 137.6ms
Speed: 2.0ms preprocess, 137.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 1 stop sign, 161.6ms
Speed: 2.0ms preprocess, 161.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 1 stop sign, 159.5ms
Speed: 2.0ms preprocess, 159.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 146.5ms
Speed: 2.0ms preprocess, 146.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 1 stop sign, 152.5ms
Speed: 2.0ms preprocess, 152.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 1 stop sign, 160.5ms
Speed: 4.0ms preprocess, 160.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 2 fire hydrants, 1 stop sign, 150.5ms
Speed: 2.0ms prepro

0: 320x640 2 cars, 126.5ms
Speed: 2.0ms preprocess, 126.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 136.1ms
Speed: 2.0ms preprocess, 136.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 161.5ms
Speed: 2.0ms preprocess, 161.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 147.5ms
Speed: 5.0ms preprocess, 147.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 150.7ms
Speed: 2.0ms preprocess, 150.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 169.5ms
Speed: 2.0ms preprocess, 169.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 174.6ms
Speed: 2.0ms preprocess, 174.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 157.0ms
Speed: 4.0ms preprocess, 157.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320

...


0: 320x640 1 car, 138.5ms
Speed: 1.0ms preprocess, 138.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 140.0ms
Speed: 2.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 164.1ms
Speed: 3.0ms preprocess, 164.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 127.1ms
Speed: 5.0ms preprocess, 127.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 152.0ms
Speed: 2.0ms preprocess, 152.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 138.5ms
Speed: 1.0ms preprocess, 138.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 161.1ms
Speed: 2.0ms preprocess, 161.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 163.1ms
Speed: 1.0ms preprocess, 163.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)



0: 320x640 1 person, 2 cars, 1 stop sign, 171.0ms
Speed: 1.0ms preprocess, 171.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 146.5ms
Speed: 2.0ms preprocess, 146.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 164.5ms
Speed: 3.0ms preprocess, 164.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 166.5ms
Speed: 3.0ms preprocess, 166.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 156.5ms
Speed: 2.0ms preprocess, 156.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 165.1ms
Speed: 2.0ms preprocess, 165.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 1 stop sign, 135.5ms
Speed: 2.0ms preprocess, 135.5ms inference, 2.0ms postprocess per


0: 320x640 1 car, 1 stop sign, 169.5ms
Speed: 5.0ms preprocess, 169.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 166.0ms
Speed: 4.0ms preprocess, 166.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 164.0ms
Speed: 2.0ms preprocess, 164.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 132.5ms
Speed: 2.0ms preprocess, 132.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 164.0ms
Speed: 2.0ms preprocess, 164.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 174.0ms
Speed: 2.0ms preprocess, 174.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 stop sign, 148.1ms
Speed: 3.0ms preprocess, 148.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 1 stop sign, 1


0: 320x640 1 person, 2 cars, 1 stop sign, 166.0ms
Speed: 2.0ms preprocess, 166.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 154.0ms
Speed: 3.0ms preprocess, 154.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 stop sign, 163.1ms
Speed: 3.0ms preprocess, 163.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 stop sign, 164.0ms
Speed: 2.0ms preprocess, 164.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 1 stop sign, 160.6ms
Speed: 2.5ms preprocess, 160.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 1 stop sign, 163.6ms
Speed: 2.0ms preprocess, 163.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 1 stop sign, 164.5ms
Speed: 5.1ms preprocess, 164.5ms inference, 2.0ms postprocess per image at 


0: 320x640 3 cars, 224.6ms
Speed: 2.0ms preprocess, 224.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 181.1ms
Speed: 3.0ms preprocess, 181.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 189.0ms
Speed: 3.0ms preprocess, 189.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 153.2ms
Speed: 3.0ms preprocess, 153.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 205.5ms
Speed: 2.0ms preprocess, 205.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
[1006, 362]

0: 320x640 3 cars, 169.0ms
Speed: 3.0ms preprocess, 169.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 stop sign, 150.0ms
Speed: 2.0ms preprocess, 150.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 stop sign, 154.0ms
Speed: 4.0ms preprocess, 154.0ms inference, 1.0ms po

In [2]:
# def RGB(event, x, y, flags, param):
#     if event == cv2.EVENT_MOUSEMOVE :  
#         colorsBGR = [x, y]
#         print(colorsBGR)
        

# cv2.namedWindow('RGB')
# cv2.setMouseCallback('RGB', RGB)

# cap=cv2.VideoCapture('veh2.mp4')


# my_file = open("coco.txt", "r")
# data = my_file.read()
# class_list = data.split("\n") 
# #print(class_list)

# count=0

# tracker=Tracker()

# cy1=322
# cy2=368

# offset=6

# vh_down={}
# counter=[]


# vh_up={}
# counter1=[]

# while True:    
#     ret,frame = cap.read()
#     if not ret:
#         break
#     count += 1
#     if count % 3 != 0:
#         continue
#     frame=cv2.resize(frame,(1020,500))
   

#     results=model.predict(frame)
#  #   print(results)
#     a=results[0].boxes.boxes
#     px=pd.DataFrame(a).astype("float")
# #    print(px)
#     list=[]
             
#     for index,row in px.iterrows():
# #        print(row)
 
#         x1=int(row[0])
#         y1=int(row[1])
#         x2=int(row[2])
#         y2=int(row[3])
#         d=int(row[5])
#         c=class_list[d]
#         if 'car' in c:
#             list.append([x1,y1,x2,y2])
#     bbox_id=tracker.update(list)
#     for bbox in bbox_id:
#         x3,y3,x4,y4,id=bbox
#         cx=int(x3+x4)//2
#         cy=int(y3+y4)//2
        
#         cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)
        


#         if cy1<(cy+offset) and cy1 > (cy-offset):
#            vh_down[id]=time.time()
#         if id in vh_down:
          
#            if cy2<(cy+offset) and cy2 > (cy-offset):
#              elapsed_time=time.time() - vh_down[id]
#              if counter.count(id)==0:
#                 counter.append(id)
#                 distance = 10 # meters
#                 a_speed_ms = distance / elapsed_time
#                 a_speed_kh = a_speed_ms * 3.6
#                 cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
#                 cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
#                 cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
#         #####going UP#####     
#         if cy2<(cy+offset) and cy2 > (cy-offset):
#            vh_up[id]=time.time()
#         if id in vh_up:

#            if cy1<(cy+offset) and cy1 > (cy-offset):
#              elapsed1_time=time.time() - vh_up[id]

 


#              if counter1.count(id)==0:
#                 counter1.append(id)      
#                 distance1 = 10 # meters
#                 a_speed_ms1 = distance1 / elapsed1_time
#                 a_speed_kh1 = a_speed_ms1 * 3.6
#                 cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
#                 cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
#                 cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

           

#     cv2.line(frame,(274,cy1),(814,cy1),(255,255,255),1)

#     cv2.putText(frame,('L1'),(277,320),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


#     cv2.line(frame,(177,cy2),(927,cy2),(255,255,255),1)
 
#     cv2.putText(frame,('L2'),(182,367),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
#     d=(len(counter))
#     u=(len(counter1))
#     cv2.putText(frame,('goingdown:-')+str(d),(60,90),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

#     cv2.putText(frame,('goingup:-')+str(u),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
#     cv2.imshow("RGB", frame)
#     if cv2.waitKey(1)&0xFF==27:
#         break
# cap.release()
# cv2.destroyAllWindows()




0: 320x640 1 person, 3 cars, 1 fire hydrant, 144.0ms
Speed: 3.0ms preprocess, 144.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)


AttributeError: 'Boxes' object has no attribute 'boxes'. See valid attributes below.

    A class for storing and manipulating detection boxes.

    Args:
        boxes (torch.Tensor | numpy.ndarray): A tensor or numpy array containing the detection boxes,
            with shape (num_boxes, 6) or (num_boxes, 7). The last two columns contain confidence and class values.
            If present, the third last column contains track IDs.
        orig_shape (tuple): Original image size, in the format (height, width).

    Attributes:
        xyxy (torch.Tensor | numpy.ndarray): The boxes in xyxy format.
        conf (torch.Tensor | numpy.ndarray): The confidence values of the boxes.
        cls (torch.Tensor | numpy.ndarray): The class values of the boxes.
        id (torch.Tensor | numpy.ndarray): The track IDs of the boxes (if available).
        xywh (torch.Tensor | numpy.ndarray): The boxes in xywh format.
        xyxyn (torch.Tensor | numpy.ndarray): The boxes in xyxy format normalized